## 3.1 Descarga de imágenes

In [1]:
import pandas as pd
import numpy  as np  
import requests
from imutils import paths
import cv2
import os
import pickle

#### Funciones Auxiliares

In [2]:
def descargar_imagenes(path, lista_imagenes, lista_id):
    
    for indice_id, url in enumerate(lista_imagenes):  
        try:
            r = requests.get(url, timeout=60)
            p = path + str (lista_id[indice_id]) + ".jpg"
            f = open(p, "wb")
            f.write(r.content)
            f.close()
        except:  
            print("[INFO] error downloading {}...skipping".format(p))
        break

In [3]:
def comprobar_imagenes(path):
    
    for imagePath in paths.list_images (path):
        try: 
            if (cv2.imread(imagePath) is None):
                print ("[INFO] deleting {}".format(imagePath))  
                os.remove (imagePath)
        except:            
            print ("[INFO] deleting {}".format(imagePath))  
            os.remove (imagePath)

#### Empezamos con train

In [4]:
train = pd.read_csv('../2.Modelado_sin_Imagenes/Data/train_cleaned.csv', sep =',', decimal='.', index_col=0) 
train.dropna(subset=['Picture Url'], inplace=True)

test = pd.read_csv('../2.Modelado_sin_Imagenes/Data/test_cleaned.csv', sep =',', decimal='.', index_col=0) 
test.dropna(subset=['Picture Url'], inplace=True)

In [5]:
lista_id_train = list (train['ID'])
lista_imagenes_train = list (train['Picture Url'])
#descargar_imagenes('img_train/',lista_imagenes_train, lista_id_train)
#comprobar_imagenes('img_train/')

#### Lo mismo para test

In [6]:
lista_id_test = list (test['ID'])
lista_imagenes_test = list (test['Picture Url'])
#descargar_imagenes('img_test/',lista_imagenes_test, lista_id_test)
#comprobar_imagenes('img_test/')

## 3.2 Dataset de imágenes y precio

A continuación voy a construir el dataset que servirá de entrada a la red covolucional. Este dataset contendrá imágenes de 50x50x3 y el precio correspondiente.

Al mismo tiempo, se crea otro dataset con los datos de airbnb siguiendo el mismo orden de carga de la estructura de imágenes.  

Estos datasets serán utilizados en la última parte de la práctica

In [7]:
def data(path,train):
    
    train_img = pd.DataFrame (columns = list(train.columns))      
    X = []
    y = []
    IMG_SIZE = 50   

    #Voy montando la estructura leyendo las imágenes desde su path
    for imagePath in paths.list_images (path):  
      try:
        img_array = cv2.imread (imagePath, cv2.IMREAD_UNCHANGED)
        if (img_array.shape[2]==3):  #Por si hay alguna imagen en blanco y negro no cogerla
            #Obtengo el ID
            a = os.path.splitext(imagePath)[0].split("/") #Divido por el caracter "/"               
            X.append(cv2.resize (img_array, (IMG_SIZE, IMG_SIZE)))
            y.append(train.loc[int(a[-1]), 'Price'])
            #Añado la linea correspodiente al nuevo dataset de datos
            train_img = train_img.append(train.loc[[int(a[-1])]])       
      except Exception as e:
        pass  
    
    return np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3), np.array(y), train_img

In [8]:
train.set_index('ID',inplace=True)
X_train,y_train,train_img = data('img_train/',train)

test.set_index('ID',inplace=True)
X_test,y_test,test_img = data('img_test/',test)

## 3.3 Guardo datasets

In [9]:
def guardar_pickle(path,data):
    pickle_out = open(path, "wb")
    pickle.dump(data, pickle_out)
    pickle_out.close()   

In [10]:
guardar_pickle('../4.Modelado_con_Imagenes/Data/X_train.pickle',X_train)
guardar_pickle('../4.Modelado_con_Imagenes/Data/y_train.pickle',y_train)
train_img.to_csv('../4.Modelado_con_Imagenes/Data/train_img.csv', index=id, header=True)

guardar_pickle('../4.Modelado_con_Imagenes/Data/X_test.pickle',X_test)
guardar_pickle('../4.Modelado_con_Imagenes/Data/y_test.pickle',y_test)
test_img.to_csv('../4.Modelado_con_Imagenes/Data/test_img.csv', index=id, header=True)